In [1]:
import numpy as np
import torch
from __future__ import print_function
from torch import nn
from torch import autograd
import torch.optim as optim
import torch.nn.functional as F
from model import TextGenerator 
from get_data import *

In [2]:
#preparing raw data
filename = "alice.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
seq_len = 50
n_chars, n_vocab, n_examples, X, Y, int_to_char, char_to_int = get_data(raw_text, seq_len)

In [3]:
#create model
H = 256 #hidden size
D = n_vocab # features
layers = 2 # number of lsmt layers
B = 8 # batch size
V = n_vocab # vocabulary size
model = TextGenerator(D,H,layers,B,V)
model.load_state_dict(torch.load('model2.txt'))
model.cuda()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.9, nesterov = True)
print (model)
loss_function = nn.CrossEntropyLoss()

RuntimeError: cuda runtime error (30) : unknown error at /home/ilya/pytorch/torch/lib/THC/THCGeneral.c:66

In [4]:
num_iter = n_examples/B

for epoch in range(1):
    hidden = model.init_hidden(B)
    for it in xrange(num_iter):
        model.zero_grad()

        train = []
        target = []
        
        inp = prepeare_X(X[it:it+B],B,seq_len, n_vocab)
        inp = inp.cuda() 
        
        target = Y[it:it+B]
        tensor1 = torch.LongTensor(target)
        m = autograd.Variable(tensor1)
        m = m.cuda()
        
        tag_scores, hidden = model(inp, hidden)
        tag_scores = tag_scores.view(B*seq_len, -1)
        m = m.view(B*seq_len)
        loss = loss_function(tag_scores, m)
        if ( it%100 == 0 ):         
            print (epoch, it , loss)
        else:
            optimizer.step()
        hidden[0].detach_()
        hidden[1].detach_()
        loss.backward(retain_variables=True)
        optimizer.step()

0 0 Variable containing:
 0.7401
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0 100 Variable containing:
 0.7913
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0 200 Variable containing:
 0.8375
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0 300 Variable containing:
 0.7926
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0 400 Variable containing:
 0.8023
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0 500 Variable containing:
 0.7674
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0 600 Variable containing:
 0.8653
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0 700 Variable containing:
 0.7263
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0 800 Variable containing:
 0.8492
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0 900 Variable containing:
 0.7971
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0 1000 Variable containing:
 0.8031
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0 1100 Variable containing:
 0.7566
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0 1200 Variable containing:
 0.8038
[torch.cuda.Floa

In [4]:
start = np.random.randint(0, len(X))
pattern = X[start]
s = ""
hidden = model.init_hidden(1)
for it in xrange(1000):

    inp = prepeare_X(pattern,1,seq_len, n_vocab)
    scores, hidden = model.forward2(inp.cuda(), hidden) #for GPU
    hidden[0].detach_()
    hidden[1].detach_()
#    scores, hidden = model.forward2(inp, hidden)
    qwer, tmp = torch.max(scores,1)
    final = np.squeeze(tmp.data.cpu().numpy()) #for GPU
#    final = np.squeeze(tmp.data.numpy())
    predicted_char = int_to_char[final[seq_len-1]]
    pattern.append(final[seq_len-1])
    pattern = pattern[1:]
    s += predicted_char

-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000
-10000

In [5]:
print (s)

at was on the top of it.

‘come, it’s very curious,’ said the caterpillar.

‘well, i’ve seen them so often, of course,’ the dormouse fell as she could, ‘it’ll be sending me on messages next!’ and she began thinking over other head down of the mock turtle.

‘and how don’t believe there’s an atom of meaning in it,’ said the king, ‘and don’t be all day about it, you know--’

‘i don’t know it was your table,’ said alice.

‘why, she,’ said the caterpillar.

‘well, i’ve seen them so often, of course,’ the dormouse fell as she could, ‘it’ll be sending me on messages next!’ and she began thinking over other head down of the mock turtle.

‘and how don’t believe there’s an atom of meaning in it,’ said the king, ‘and don’t be all day about it, you know--’

‘i don’t know it was your table,’ said alice.

‘why, she,’ said the caterpillar.

‘well, i’ve seen them so often, of course,’ the dormouse fell as she could, �


In [41]:
ModelFile = open('model2.txt', 'w')
torch.save(model.state_dict(), ModelFile)